In [1]:
# voting classifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import VotingClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, noise=0.15)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [2]:
log_clf = LogisticRegression()
rnd_clf = RandomForestClassifier()
svm_clf = SVC()

In [3]:
voting_clf = VotingClassifier( estimators=[('lr',log_clf),('rf',rnd_clf),('svc',svm_clf)], voting ='hard')
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression()),
                             ('rf', RandomForestClassifier()), ('svc', SVC())])

In [4]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.76
RandomForestClassifier 0.92
SVC 0.92
VotingClassifier 0.88


In [5]:
from sklearn.ensemble import BaggingClassifier # samplea con reemplazo al training set
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier( DecisionTreeClassifier(), n_estimators=500, max_samples=10, bootstrap=True, n_jobs=-1, oob_score=True)
bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [6]:
bag_clf.oob_score_ # out of the bag deja samples sin entrenar, oob_score compara la performance contra estos

0.88

In [7]:
accuracy_score(y_test, y_pred) # del sckit accuracy_score


0.88

In [8]:
# random forest classifier, utiliza bagging y samplea algunas features al azar
from sklearn.ensemble import RandomForestClassifier
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, n_jobs=-1)

rnd_clf.fit(X_train, y_train)
y_pred_rf = rnd_clf.predict(X_test) 
accuracy_score(y_test, y_pred_rf) # del sckit accuracy_score


0.92

In [9]:
# random forest makes easy to understand feature importance
from sklearn.datasets import load_iris
iris = load_iris()

rnd_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1)

rnd_clf.fit(iris["data"], iris["target"])

for name,score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.1064859309348292
sepal width (cm) 0.025163038726486342
petal length (cm) 0.4196211132055762
petal width (cm) 0.44872991713310834


In [10]:
# boosting- train using weak methods 
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier( DecisionTreeClassifier(max_depth=1), n_estimators=200, algorithm="SAMME.R", learning_rate=0.5)
ada_clf.fit(X_train, y_train)
y_pred_ada = ada_clf.predict(X_test)

In [11]:
accuracy_score(y_test, y_pred_rf)

0.92

In [12]:
# gradient boosting - sum up all predictions noise
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2) 
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2)

In [13]:
y2 = y - tree_reg1.predict(X)
tree_reg2 = DecisionTreeRegressor(max_depth=2)
tree_reg2.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [14]:
y3 = y - tree_reg2.predict(X)
tree_reg3 = DecisionTreeRegressor(max_depth=2)
tree_reg3.fit(X,y2)

DecisionTreeRegressor(max_depth=2)

In [15]:
import numpy as np

y_pred = sum(tree.predict(X_test) for tree in (tree_reg1, tree_reg2, tree_reg3))
y_pred

array([ 1.08163265,  0.90516206,  1.08163265,  1.08163265,  1.08163265,
        1.08163265,  0.90516206,  0.90516206, -0.30612245,  0.90516206,
        0.01005053,  0.90516206,  0.01005053,  1.08163265,  0.01005053,
        0.01005053,  0.98679472,  1.08163265,  0.01005053,  0.01005053,
        1.08163265,  0.01005053,  1.08163265,  1.08163265,  1.08163265])

In [16]:
from sklearn.ensemble import GradientBoostingRegressor
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0)
gbrt.fit(X,y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3)

In [17]:
import numpy as np 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y)
gbrt= GradientBoostingRegressor(max_depth=2, n_estimators=120) 

gbrt.fit(X_train, y_train)
errors = [mean_squared_error(y_val, y_pred) for y_pred in gbrt.staged_predict(X_val)]#busca el nro optimo de trees
bst_n_estimators = np.argmin(errors) + 1

gbrt_best = GradientBoostingRegressor(max_depth=2,n_estimators=bst_n_estimators)#entrena el nuevo tree con mejor estimator

gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=28)

In [21]:
# se puede saltear este paso aplicando warm_start=True, lo que indica que frena cuando reduce el error
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True)

min_val_error = float("inf") 
error_going_up = 0 
for n_estimators in range(1, 120): 
    gbrt.n_estimators = n_estimators 
    gbrt.fit(X_train, y_train) 
    y_pred = gbrt.predict(X_val) 
    val_error = mean_squared_error(y_val, y_pred) 
    if val_error < min_val_error: 
        min_val_error = val_error 
        error_going_up = 0
    else: 
        error_going_up += 1 
        if error_going_up == 5:
            break # early stopping
                


In [24]:
#XGBoost - optimize gradient booster
import xgboost
xgb_reg = xgboost.XGBRegressor()
xgb_reg.fit(X_train,y_train)
y_pred = xgb_reg.predict(X_test)
#automatic early stopping
xgb_reg.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_test)

[0]	validation_0-rmse:0.38241
[1]	validation_0-rmse:0.28669
[2]	validation_0-rmse:0.21307
[3]	validation_0-rmse:0.15968
[4]	validation_0-rmse:0.12106
[5]	validation_0-rmse:0.09327
[6]	validation_0-rmse:0.07602
[7]	validation_0-rmse:0.06055
[8]	validation_0-rmse:0.05240
[9]	validation_0-rmse:0.04860
[10]	validation_0-rmse:0.04205
[11]	validation_0-rmse:0.04075
[12]	validation_0-rmse:0.03729
[13]	validation_0-rmse:0.03313
[14]	validation_0-rmse:0.03023
[15]	validation_0-rmse:0.02802
[16]	validation_0-rmse:0.02538
[17]	validation_0-rmse:0.02315
[18]	validation_0-rmse:0.02159
[19]	validation_0-rmse:0.02071
[20]	validation_0-rmse:0.02065
[21]	validation_0-rmse:0.01983
[22]	validation_0-rmse:0.01967
[23]	validation_0-rmse:0.01878
[24]	validation_0-rmse:0.01829
[25]	validation_0-rmse:0.01814
[26]	validation_0-rmse:0.01789
[27]	validation_0-rmse:0.01776
[28]	validation_0-rmse:0.01759
[29]	validation_0-rmse:0.01747
[30]	validation_0-rmse:0.01732
[31]	validation_0-rmse:0.01719
[32]	validation_0-

In [ ]:
#STACKING- blending predictor (meta predictor, takes all predictors and train the data set with it)


